# Building a Report Picker eval

Based on the [examples in openai/evals](https://github.com/openai/evals/tree/26a5191230e91953f4f2fdd74a82051fda2aba23/examples)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import requests
import pandas as pd

In [13]:
# Construct the system prompt, similar to ../lib/openai.ts `createSystemPrompt`
import json

# Read the prompt_template as a scalar value
with open("../lib/prompt_template.json") as f:
    prompt_template_data = json.load(f)
    prompt_template_prompt = prompt_template_data['prompt'] # Adjust this based on the actual structure

# Read the available_reports as a DataFrame
with open("../lib/available_reports/square.json") as f:
    available_reports = json.load(f)

sample_date = "2023-08-21"

system_prompt = f"""
{prompt_template_prompt}

Today's date: {sample_date}

Available reports: 
{available_reports}
"""

system_prompt

'\nYou\'re an expert at finding the best report(s) to answer a user\'s question about their business. If the question isn\'t answerable by these reports, return "NA" as the "error".\n\nToday\'s date: 2023-08-21\n\nAvailable reports: \n[{\'key\': \'SalesSummary\', \'description\': \'Sales Summary: A general overview of sales from a given time period. The report includes itemization metrics like gross sales, refunds, net sales, discounts, tips, and taxes. It also includes payment metrics like total collected (revenue by tender type) and fees.\', \'filters\': [\'startDate\', \'endDate\']}, {\'key\': \'SalesTrends\', \'description\': \'Sales Trends: Compares your daily, weekly, and yearly gross sales.\', \'filters\': [\'startDate\', \'endDate\']}, {\'key\': \'PaymentMethods\', \'description\': \'Payment Methods: A summary of the total collected and any associated fees from credit, debit and gift cards, and any other tender types. The reports include payments broken down by debit vs credit,

In [11]:
def create_chat_prompt(text):
    return [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": text}
    ]